<a href="https://colab.research.google.com/github/brettmorrisonAO5ANNEX/Super-Resolution-Implementation/blob/main/SRCNN_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from data import DIV2K

# Data Preprocessing

## Load Training Data

In [ ]:
train_loader = DIV2K(scale=4,             # 2, 3, 4 or 8
                     downgrade='bicubic', # 'bicubic', 'unknown', 'mild' or 'difficult'
                     subset='train')      # Training dataset are images 001 - 800

# Create a tf.data.Dataset
train_ds = train_loader.dataset(batch_size=16,         # batch size as described in the EDSR and WDSR papers
                                random_transform=True, # random crop, flip, rotate as described in the EDSR paper
                                repeat_count=None)     # repeat iterating over training images indefinitely

## Load Validation Data

In [ ]:
valid_loader = DIV2K(scale=4,             # 2, 3, 4 or 8
                     downgrade='bicubic', # 'bicubic', 'unknown', 'mild' or 'difficult'
                     subset='valid')      # Validation dataset are images 801 - 900

# Create a tf.data.Dataset
valid_ds = valid_loader.dataset(batch_size=1,           # use batch size of 1 as DIV2K images have different size
                                random_transform=False, # use DIV2K images in original size
                                repeat_count=1)         # 1 epoch

# Creating The Model

In [ ]:
def SRCNN():
    model = Sequential()
    # Patch extraction and representation
    model.add(Conv2D(64, (9, 9), activation='relu', padding='same', input_shape=(None, None, 3)))
    # Non-linear mapping
    model.add(Conv2D(32, (1, 1), activation='relu', padding='same'))
    # Reconstruction
    model.add(Conv2D(3, (5, 5), activation='linear', padding='same'))
    return model

srcnn = SRCNN()
srcnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Training & Evaluation

In [ ]:
history = srcnn.fit(train_dataset, epochs=50, validation_data=val_dataset)

In [ ]:
loss, accuracy = srcnn.evaluate(val_dataset)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")